
[Use the link for reference as guideline](https://realpython.com/sentiment-analysis-python/#vectorizing-text)


Using Natural Language Processing to Preprocess and Clean Text Data¶ <br>
1-Tokenizing <br>
2-Removing Stop Words<br>
3-Normalizing Words<br>
4-Vectorizing Text<br>
5-Using Machine Learning Classifiers to Predict Sentiment<br>
6-Machine Learning Tools<br>

How Classification Works 7-How to Use spaCy for Text Classification<br>

8-Building Your Own NLP Sentiment Analyzer Loading and Preprocessing Data<br> Training Your Classifier Classifying Reviews Connecting the Pipeline Conclusion<br>

https://realpython.com/sentiment-analysis-python/     <br>

1.clean data<br>
2.data manipulation<br>
3.merge dataset<br>
4.sentiment analysis on review_content, compare with Harvard dict, positive and negative dict, two column with number of pos and negs. including negation.compare pos-neg=difference#1, then we rank the difference value,then use value to create a recommonded list and do-not-watch list.<br>

6.Mannually make a film dictionary first. How to create the dictionary???? <br>
7.craete new column called 'tags' ,with tokenzied 'movie term keyword'<br>


In [ ]:
from google.colab import drive
drive.mount('/content/drive') 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import re
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Load the data

In [ ]:
%%capture
%pylab inline

In [ ]:
df1= pd.read_csv('/content/drive/Shareddrives/NLP Group Project/rotten_tomatoes_critic_reviews.csv')
df2= pd.read_csv('/content/drive/Shareddrives/NLP Group Project/rotten_tomatoes_movies.csv')

In [ ]:
#Load Harvard Inquirer Dictionary
f = open('/content/drive/Shareddrives/NLP Group Project/inqdict.txt')
HIDict = f.read()
HIDict = HIDict.splitlines()
HIDict[:20]

['Entryword Source Pos Neg Pstv Affil Ngtv Hostile Strng Power Weak Subm Actv Psv Pleasure Pain Arousal EMOT Feel Virtue Vice Ovrst Undrst Acad Doctr Econ* Exch ECON Exprs Legal Milit Polit* POLIT Relig Role COLL Work Ritual Intrel Race Kin* MALE Female Nonadlt HU ANI PLACE Social Region Route Aquatic Land Sky Object Tool Food Vehicle Bldgpt Natobj Bodypt Comnobj Comform COM Say Need Goal Try Means Ach Persist Complt Fail Natpro Begin Vary Change Incr Decr Finish Stay Rise Move Exert Fetch Travel Fall Think Know Causal Ought Percv Comp Eval EVAL Solve Abs* ABS Qual Quan NUMB ORD CARD FREQ DIST Time* TIME Space POS DIM Dimn Rel COLOR Self Our You Name Yes No Negate Intrj IAV DAV SV IPadj IndAdj POWGAIN POWLOSS POWENDS POWAREN POWCON POWCOOP POWAPT POWPT POWDOCT POWAUTH POWOTH POWTOT RCTETH RCTREL RCTGAIN RCTLOSS RCTENDS RCTTOT RSPGAIN RSPLOSS RSPOTH RSPTOT AFFGAIN AFFLOSS AFFPT AFFOTH AFFTOT WLTPT WLTTRAN WLTOTH WLTTOT WLBGAIN WLBLOSS WLBPHYS WLBPSYC WLBPT WLBTOT ENLGAIN ENLLOSS ENLENDS

In [ ]:
HIDict = HIDict[1:]
print(HIDict[:5])
print(len(HIDict))
poswords = [j for j in HIDict if "Pos" in j]  #using a list comprehension
poswords = [j.split()[0] for j in poswords]
poswords = [j.split("#")[0] for j in poswords]
poswords = unique(poswords)
poswords = [j.lower() for j in poswords]
print(poswords[:20])
print(len(poswords))

['A H4Lvd DET ART  | article: Indefinite singular article--some or any one', 'ABANDON H4Lvd Neg Ngtv Weak Fail IAV AFFLOSS AFFTOT SUPV  |', 'ABANDONMENT H4 Neg Weak Fail Noun  |', 'ABATE H4Lvd Neg Psv Decr IAV TRANS SUPV  |', 'ABATEMENT Lvd Noun  ']
11895
['abide', 'able', 'abound', 'absolve', 'absorbent', 'absorption', 'abundance', 'abundant', 'accede', 'accentuate', 'accept', 'acceptable', 'acceptance', 'accessible', 'accession', 'acclaim', 'acclamation', 'accolade', 'accommodate', 'accommodation']
1646


In [ ]:
#Extract all the lines that contain the Neg tag
negwords = [j for j in HIDict if "Neg" in j]  #using a list comprehension
negwords = [j.split()[0] for j in negwords]
negwords = [j.split("#")[0] for j in negwords]
negwords = unique(negwords)
negwords = [j.lower() for j in negwords]
print(negwords[:20])
print(len(negwords))

['abandon', 'abandonment', 'abate', 'abdicate', 'abhor', 'abject', 'abnormal', 'abolish', 'abominable', 'abrasive', 'abrupt', 'abscond', 'absence', 'absent', 'absent-minded', 'absentee', 'absurd', 'absurdity', 'abuse', 'abyss']
2120


In [ ]:
#Need to change accordingly
def movieScore(url,poswords,negwords):
    f = requests.get(url)
    text = f.text
    f.close()
    text = BeautifulSoup(text,'lxml').get_text()    
    text = text.replace('\n',' ').lower()
    text = text.split(' ')
    posmatches = set(text).intersection(set(poswords))
    print(posmatches)
    print(len(posmatches))
    negmatches = set(text).intersection(set(negwords))
    print(negmatches)
    print(len(negmatches))

### Data Cleaning

In [ ]:
#We compare the total missing value on review_score
df1.review_score.isna().sum()


305936

In [ ]:
df1.review_score.notna().sum()# so we decide to fill missing value with 0

824081

In [ ]:
df1.review_score.dropna(inplace= True)

In [ ]:
df1.review_score.isna().sum()

305936

In [ ]:
#We choose the top critic review, after year 2000, and fill missing movie_score with 0
df1 = df1[(df1.top_critic==True) & (df1.review_date>'2000-01-01')] #run only once


In [ ]:
df1 = df1[['rotten_tomatoes_link', 'critic_name', 'review_type', 'review_content']]
df1.head()

,rotten_tomatoes_link,critic_name,review_type,review_content
4,m/0814255,Ethan Alter,Rotten,What's really lacking in The Lightning Thief i...
5,m/0814255,David Germain,Rotten,It's more a list of ingredients than a movie-m...
7,m/0814255,Bill Goodykoontz,Fresh,"Percy Jackson isn't a great movie, but it's a ..."
9,m/0814255,Jim Schembri,Fresh,"Crammed with dragons, set-destroying fights an..."
11,m/0814255,Roger Moore,Rotten,"For what it is and for whom it is intended, it..."


In [ ]:
#Checking for null values in df1
print(df1.isna().sum())
print("The total records of df1 is", len(df1))

rotten_tomatoes_link       0
critic_name             3412
review_type                0
review_content          5114
dtype: int64
The total records of df1 is 271021


In [ ]:
#review_content column cleaning
df1.review_content = df1.review_content.str.lower()
df1.review_type = df1.review_type.str.lower()

In [ ]:
df2['tomatometer_avg_rating']=df2.tomatometer_rating/df2.tomatometer_count*100
df2['tomatometer_avg_rating'].describe()

count    17668.000000
mean       327.754111
std        384.107637
min          0.000000
25%         59.128848
50%        183.333333
75%        428.571429
max       2000.000000
Name: tomatometer_avg_rating, dtype: float64

In [ ]:
df2['audience_avg_rating'] = df2.audience_rating/df2.audience_count*10000
df2['audience_avg_rating'].describe()

count     17415.000000
mean       2030.582934
std        9981.481919
min           0.000000
25%          23.313616
50%         137.733142
75%         764.518546
max      200000.000000
Name: audience_avg_rating, dtype: float64

In [ ]:
df2 = df2[['rotten_tomatoes_link', 'movie_title', 'content_rating','tomatometer_avg_rating','audience_avg_rating', 'genres', 'directors', 'actors']]
df2.head()

,rotten_tomatoes_link,movie_title,content_rating,tomatometer_avg_rating,audience_avg_rating,genres,directors,actors
0,m/0814255,Percy Jackson & the Olympians: The Lightning T...,PG,32.885906,2.083161,"Action & Adventure, Comedy, Drama, Science Fic...",Chris Columbus,"Logan Lerman, Brandon T. Jackson, Alexandra Da..."
1,m/0878835,Please Give,R,61.267606,55.296354,Comedy,Nicole Holofcener,"Catherine Keener, Amanda Peet, Oliver Platt, R..."
2,m/10,10,R,279.166667,36.093707,"Comedy, Romance",Blake Edwards,"Dudley Moore, Bo Derek, Julie Andrews, Robert ..."
3,m/1000013-12_angry_men,12 Angry Men (Twelve Angry Men),NR,185.185185,9.204259,"Classics, Drama",Sidney Lumet,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G...."
4,m/1000079-20000_leagues_under_the_sea,"20,000 Leagues Under The Sea",G,329.629630,10.737398,"Action & Adventure, Drama, Kids & Family",Richard Fleischer,"James Mason, Kirk Douglas, Paul Lukas, Peter L..."


In [ ]:
#Checking for null values in df2
df2.isna().sum()

rotten_tomatoes_link        0
movie_title                 0
content_rating              0
tomatometer_avg_rating     44
audience_avg_rating       297
genres                     19
directors                 194
actors                    352
dtype: int64

In [ ]:
df2['audience_avg_rating'].dropna(inplace=True)
df2['tomatometer_avg_rating'].dropna(inplace=True)

In [ ]:
#Merge the two datasets
df = pd.merge(df2, df1, right_on='rotten_tomatoes_link', left_on='rotten_tomatoes_link', how='left')
df.drop('rotten_tomatoes_link', axis=1, inplace=True)

In [ ]:
df.to_csv(r'/content/drive/Shareddrives/NLP Group Project/movie_clean.csv', index = False)

## Data Manipulation/Prep for Analysis

In [ ]:
import pandas as pd 
import numpy as np
df =pd.read_csv('/content/drive/Shareddrives/NLP Group Project/movie_clean.csv')

In [ ]:
print(len(df))

272854


In [ ]:
df.dropna(inplace=True)
len(df)

256851

In [ ]:
df.head()

,movie_title,content_rating,tomatometer_avg_rating,audience_avg_rating,genres,directors,actors,critic_name,review_type,review_content
0,Percy Jackson & the Olympians: The Lightning T...,PG,32.885906,2.083161,"Action & Adventure, Comedy, Drama, Science Fic...",Chris Columbus,"Logan Lerman, Brandon T. Jackson, Alexandra Da...",Ethan Alter,rotten,what's really lacking in the lightning thief i...
1,Percy Jackson & the Olympians: The Lightning T...,PG,32.885906,2.083161,"Action & Adventure, Comedy, Drama, Science Fic...",Chris Columbus,"Logan Lerman, Brandon T. Jackson, Alexandra Da...",David Germain,rotten,it's more a list of ingredients than a movie-m...
2,Percy Jackson & the Olympians: The Lightning T...,PG,32.885906,2.083161,"Action & Adventure, Comedy, Drama, Science Fic...",Chris Columbus,"Logan Lerman, Brandon T. Jackson, Alexandra Da...",Bill Goodykoontz,fresh,"percy jackson isn't a great movie, but it's a ..."
3,Percy Jackson & the Olympians: The Lightning T...,PG,32.885906,2.083161,"Action & Adventure, Comedy, Drama, Science Fic...",Chris Columbus,"Logan Lerman, Brandon T. Jackson, Alexandra Da...",Jim Schembri,fresh,"crammed with dragons, set-destroying fights an..."
4,Percy Jackson & the Olympians: The Lightning T...,PG,32.885906,2.083161,"Action & Adventure, Comedy, Drama, Science Fic...",Chris Columbus,"Logan Lerman, Brandon T. Jackson, Alexandra Da...",Roger Moore,rotten,"for what it is and for whom it is intended, it..."


In [ ]:
#Tokenize each records
df['genres']= df['genres'].str.replace(r'[^\w\s]+', '')
df['directors']= df['directors'].str.replace(r'[^\w\s]+', '')

In [ ]:
#Give value 1 and 0 to rotten and fresh in review_type 
df = df.replace({'review_type': {'fresh': 1, 
                                'rotten': 0}})

In [ ]:
#tokenize review_content
#remove punctuation
def removePuncStr(s):
    for c in string.punctuation:
        s = s.replace(c," ")
    return s

def removePunc(text_array):
    return [removePuncStr(h) for h in text_array]

In [ ]:
#remove numbers
def removeNumbersStr(s):
    for c in range(10):
        n = str(c)
        s = s.replace(n," ")
    return s

def removeNumbers(text_array):
    return [removeNumbersStr(h) for h in text_array]

In [ ]:
#remove stopwords
def stopText(text_array):
    stop_words = set(stopwords.words('english'))
    stopped_text = []
    for h in text_array:
        words = word_tokenize(h)
        h2 = ''
        for w in words:
            if w.lower() not in stop_words:
                h2 = h2 + ' ' + w
        stopped_text.append(h2)
    return stopped_text

In [ ]:
df.dtypes

movie_title                object
content_rating             object
tomatometer_avg_rating    float64
audience_avg_rating       float64
genres                     object
directors                  object
actors                     object
critic_name                object
review_type                 int64
review_content             object
dtype: object

In [ ]:
df['review_content']=removePunc(df['review_content'])

In [ ]:
df['review_content']=removeNumbers(df['review_content'])

In [ ]:
df['review_content']=stopText(df['review_content'])

In [ ]:
df.isna().sum()

movie_title               0
content_rating            0
tomatometer_avg_rating    0
audience_avg_rating       0
genres                    0
directors                 0
actors                    0
critic_name               0
review_type               0
review_content            0
dtype: int64

In [ ]:
df.columns

Index(['movie_title', 'content_rating', 'tomatometer_avg_rating',
       'audience_avg_rating', 'genres', 'directors', 'actors', 'critic_name',
       'review_type', 'review_content'],
      dtype='object')

In [ ]:
df=df[['movie_title','review_type','content_rating','genres', 'directors', 'actors', 'critic_name', 'tomatometer_avg_rating', 'audience_avg_rating' ,'review_content']]

In [ ]:
#df['keywords']=df['actors']+ " " + df['critic_name']+ " " + df['directors']+ " " + df['genres']+ " " + df['content_rating']+ " " + df['review_content']

In [ ]:
#df.drop(columns=['content_rating','genres', 'directors', 'actors', 'critic_name',
#        'review_content'],inplace= True)
df.drop(columns=['review_type','critic_name','audience_avg_rating'],inplace= True)


In [ ]:
df.head()

,movie_title,content_rating,genres,directors,actors,tomatometer_avg_rating,review_content
0,Percy Jackson & the Olympians: The Lightning T...,PG,Action Adventure Comedy Drama Science Fiction...,Chris Columbus,"Logan Lerman, Brandon T. Jackson, Alexandra Da...",32.885906,really lacking lightning thief genuine sense ...
1,Percy Jackson & the Olympians: The Lightning T...,PG,Action Adventure Comedy Drama Science Fiction...,Chris Columbus,"Logan Lerman, Brandon T. Jackson, Alexandra Da...",32.885906,list ingredients movie magic potion enjoy sta...
2,Percy Jackson & the Olympians: The Lightning T...,PG,Action Adventure Comedy Drama Science Fiction...,Chris Columbus,"Logan Lerman, Brandon T. Jackson, Alexandra Da...",32.885906,percy jackson great movie good one trotting k...
3,Percy Jackson & the Olympians: The Lightning T...,PG,Action Adventure Comedy Drama Science Fiction...,Chris Columbus,"Logan Lerman, Brandon T. Jackson, Alexandra Da...",32.885906,crammed dragons set destroying fights things ...
4,Percy Jackson & the Olympians: The Lightning T...,PG,Action Adventure Comedy Drama Science Fiction...,Chris Columbus,"Logan Lerman, Brandon T. Jackson, Alexandra Da...",32.885906,intended bad movie indifferent one


In [ ]:
df.to_csv(r'/content/drive/Shareddrives/NLP Group Project/movie_tokenized.csv', index = False)

## Create Machine Learning Model-1 Jaccard 



In [ ]:
#df3 is the tokenized movie review excel file
#df4 is the film term word-list
df = pd.read_csv('/content/drive/Shareddrives/NLP Group Project/movie_tokenized.csv')
df4 = pd.read_csv('/content/drive/Shareddrives/NLP Group Project/movie_dict.csv')

In [ ]:
df[df.review_content.str.find('cgi') != -1]['review_content']

24                  cgi sorcery harry potter pic none magic
32         unfortunately movie lacks good old potter cha...
104        mix vast cgi spectacle small silly moments pr...
108        much delayed film cries consideration worst c...
126        would emmerich took much care human character...
                                ...                        
253570     overlong minutes suffers much origin story ex...
253574     final big action sequence seems always case m...
253953     cgi eye candy intense little way layers compl...
254618     climax biggest letdown giant hash crosscuttin...
256646     time film reaches abrupt conclusion complete ...
Name: review_content, Length: 485, dtype: object

In [ ]:
movies=movies.groupby('movie_title').agg(lambda a: ",".join(a)).reset_index()

In [ ]:
movies.head()

In [ ]:
movies.replace(',','',regex=True,inplace=True)

In [ ]:
movies.keywords[1]

In [ ]:
def get_jaccard_sim(str1, str2):
    a = set(str1.split(' '))
    b = set(str2.split(' '))
    c = a.intersection(b)
    return(float(len(c)) / (len(a) + len(b) - len(c)))

def jaccard_recommender(movie_title, number_of_hits=5):
    movie = movies[movies.movie_title==movie_title]
    keyword_string = movie.keywords.iloc[0]

    jaccards = []
    for movie in movies['keywords']:
        jaccards.append(get_jaccard_sim(keyword_string, movie))
    jaccards = pd.Series(jaccards)
    jaccards_index = jaccards.nlargest(number_of_hits+1).index
    matches = movies.loc[jaccards_index]
    for match,score in zip(matches['movie_title'][1:],jaccards[jaccards_index][1:]) :
        print(match,score )

In [ ]:
jaccard_recommender('Despicable Me')

In [ ]:
jaccard_recommender('Despicable Me 2')

In [ ]:
jaccard_recommender('Shrek 2')

## Create Machine Learning Model-2 TF-IDF

In [ ]:
!pip install gensim

In [ ]:
keywords = movies['keywords'].tolist()

from nltk.tokenize import word_tokenize
keywords = [word_tokenize(keyword.lower()) for keyword in keywords]

def no_commas(doc):
    no_commas = [t for t in doc if t!=',']
    return(no_commas)

keywords = [no_commas(kw) for kw in keywords]
processed_keywords = keywords

In [ ]:
from gensim.corpora.dictionary import Dictionary
dictionary = Dictionary(processed_keywords) # create a dictionary of words from our keywords

corpus = [dictionary.doc2bow(doc) for doc in processed_keywords] 
#create corpus where the corpus is a bag of words for each document

from gensim.models.tfidfmodel import TfidfModel
tfidf = TfidfModel(corpus) #create tfidf model of the corpus

from gensim.similarities import MatrixSimilarity
# Create the similarity data structure. This is the most important part where we get the similarities between the movies.
sims = MatrixSimilarity(tfidf[corpus], num_features=len(dictionary))

In [ ]:
def keywords_recommendation(keywords, number_of_hits):
    keywords=map(str.lower, keywords)
    query_doc_bow = dictionary.doc2bow(keywords)
    query_doc_tfidf = tfidf[query_doc_bow]

    similarity_array = sims[query_doc_tfidf]
    similarity_series = pd.Series(similarity_array.tolist(), index=movies.movie_title.values)
    top_hits = similarity_series.sort_values(ascending=False)[:number_of_hits] 
    # i.e. most similar movies

    # Print the top matching movies
    print("Our top %s most similar movies for the keywords %s are:" %(number_of_hits, keywords))
    for idx, (movie,score) in enumerate(zip(top_hits.index, top_hits)):
        print("%d '%s' with a similarity score of %.3f" %(idx+1, movie, score))

In [ ]:
keywords_recommendation(['Emma Watson'], 5)

In [ ]:
keywords_recommendation(['Emma', 'Watson'], 5)

# Collect User Input for Movie Search

In [ ]:
user_genre = input("What genre of movies are you looking for?\nFor example, say something like 'Horror' or 'Action'")
print(user_genre)

In [ ]:
user_director = input("Are you looking to watch a movie with a certain director?\nIf so enter their first and last name with no abbreviations. If not, type 'No'")
print(user_director)

In [ ]:
user_actor = input("Are you looking to watch a movie with a certain actor/actress?\nIf so enter their first and last name with no abbreviations. If not, type 'No'")
print(user_actor)

In [ ]:
user_keyword = input("Okay, so what are you looking for in a " + user_genre + " movie?\nSay something like 'dragon' or 'political'")
user_keyword

## Initial Filtering Based on User's Inputs

In [ ]:
#check user inputted genre
initial_filter = df[df['genres'].str.contains(user_genre)]

#check user inputted director name
if user_director != 'No':
  initial_filter = initial_filter[initial_filter['directors'].str.contains(user_director)]

#check user inputted actor name
if user_actor != 'No':
  initial_filter = initial_filter[initial_filter['actors'].str.contains(user_actor)]

#check user inputted keyword
if user_keyword != 'No':
  initial_filter = initial_filter[initial_filter['review_content'].str.contains(user_keyword)]

In [ ]:
#Concatenate duplicate rows but save keywords from content_review
initial_filter = initial_filter.groupby('movie_title').agg({'content_rating':'first', 'genres':'first','directors':'first','actors':'first','tomatometer_avg_rating':'first','review_content': ' '.join})
initial_filter = initial_filter.reset_index()
initial_filter

#If list is longer than 5? or 10?, then prompt user for another parameter filter like content_rating, or another additional genre, or another actor, or another keyword
#if len(initial_filter.movie_title) > 7:
#  print("More than 5 movies...")
#else:
#    print("Good amount of movies")

In [ ]:
print("Here are your top recommended movies to watch:\n")
print(initial_filter.movie_title.to_string(index=False))
user_similar = input("\nSee a movie you've already seen on the list and enjoyed watching it?\nPaste the movie title here and we'll help you find a similar movie or type 'No':")

In [ ]:
user_similar = user_similar.strip()

In [ ]:
jaccard_recommender(user_similar)

## Data analysis and Movie-search Engine prototype


In [ ]:
sns.countplot(df3.review_type)

In [ ]:
sns.countplot(df.content_rating)